In [3]:
from neo4j import GraphDatabase
import time

In [4]:
# Get Connection to neo4j

uri = 'bolt://localhost:7687'
username = 'neo4j'
password = 'graph'

neo4j_driver = GraphDatabase.driver(uri, auth=(username,password))

In [ ]:
# DATA IS LOADED USING neo4j-import

#Netflow = "LOAD CSV WITH HEADERS FROM 'file:///nf_200.csv' AS row MERGE (m:Devices {Name:row.src_device}) MERGE (n:Devices {Name:row.dst_device}) MERGE (m) -[:Netflow { epoch_time:toInteger(row.epoch_time), duration:toInteger(row.duration), protocol:toInteger(row.protocol), src_port:toInteger(row.src_port), dst_port:toInteger(row.dst_port), src_packets:toInteger(row.src_packets), dst_packets:toInteger(row.dst_packets), src_bytes:toInteger(row.src_bytes), dst_bytes:toInteger(row.dst_bytes) }]-> (n)"

#start = time.time()
#with neo4j_driver.session() as neo4j_Session:
#    neo4j_Session.run(Netflow)

In [ ]:
# DATA IS LOADED USING neo4j-import


#HostEvents = "LOAD CSV WITH HEADERS FROM 'file:///1v_200.csv' AS row MERGE (m:Devices {Name:row.log_host}) MERGE (m) -[:HostEvents { epoch_time:toInteger(row.epoch_time),event_id:toInteger(row.event_id),user_name:row.user_name, domain_name:row.domain_name, logon_id: toInteger(row.logon_id), process_name:row.process_name, process_id:toInteger(row.process_id), parent_process_name:row.parent_process_name, parent_process_id:toInteger(row.parent_process_id) }]-> (m)" 
                         
#start = time.time()
#with neo4j_driver.session() as neo4j_Session:
#    neo4j_Session.run(HostEvents)

In [5]:
Port_Query = "MATCH (v0:Devices)-[edge:Netflow]->(v1:Devices) WHERE edge.dst_port=135 OR edge.dst_port=445 CREATE (v0)-[e:LMFlow {epoch_time : toInteger(edge.epoch_time), duration : toInteger(edge.duration), protocol : toInteger(edge.protocol), src_port : toInteger(edge.src_port), dst_port : toInteger(edge.dst_port)}]->(v1) RETURN count(*)"


with neo4j_driver.session() as neo4j_Session:
    result1 = neo4j_Session.run(Port_Query)

print(result1.data())

[{'count(*)': 5210584}]


In [6]:
LMflow_Query = "MATCH (A:Devices)-[r1:LMFlow]->(B:Devices) MATCH (A)-[hijack1:HostEvents]->(A) WHERE A <> B AND hijack1.event_id = 4688 AND hijack1.parent_process_name = 'lsass' AND hijack1.epoch_time <= r1.epoch_time AND r1.epoch_time - hijack1.epoch_time < 180 RETURN A, B, r1.epoch_time, hijack1.epoch_time"


with neo4j_driver.session() as neo4j_Session:
    result2 = neo4j_Session.run(LMflow_Query)

print(result2.data())

[{'A': <Node id=42573 labels={'Devices'} properties={'src': 'Comp921148'}>, 'B': <Node id=31481 labels={'Devices'} properties={'src': 'ActiveDirectory'}>, 'r1.epoch_time': 7290235, 'hijack1.epoch_time': 7290114}, {'A': <Node id=42573 labels={'Devices'} properties={'src': 'Comp921148'}>, 'B': <Node id=31481 labels={'Devices'} properties={'src': 'ActiveDirectory'}>, 'r1.epoch_time': 7290154, 'hijack1.epoch_time': 7290114}, {'A': <Node id=42573 labels={'Devices'} properties={'src': 'Comp921148'}>, 'B': <Node id=31481 labels={'Devices'} properties={'src': 'ActiveDirectory'}>, 'r1.epoch_time': 7290140, 'hijack1.epoch_time': 7290114}, {'A': <Node id=42573 labels={'Devices'} properties={'src': 'Comp921148'}>, 'B': <Node id=31481 labels={'Devices'} properties={'src': 'ActiveDirectory'}>, 'r1.epoch_time': 7290201, 'hijack1.epoch_time': 7290114}, {'A': <Node id=42573 labels={'Devices'} properties={'src': 'Comp921148'}>, 'B': <Node id=31481 labels={'Devices'} properties={'src': 'ActiveDirectory'}